In [1]:
import torch.utils.data
import sklearn.metrics
import torch
import pandas
from torch.utils.data import Dataset
import tqdm
import spacy
import numpy as np
nlp = spacy.load('en')
pandas.read_csv('train.csv',header=0).iloc[0]

id                                                          1
keyword                                                   NaN
location                                                  NaN
text        Our Deeds are the Reason of this #earthquake M...
target                                                      1
Name: 0, dtype: object

In [2]:
pandas.read_csv('test.csv',header=0).iloc[0]

id                                           0
keyword                                    NaN
location                                   NaN
text        Just happened a terrible car crash
Name: 0, dtype: object

In [3]:
class TrainDataset(Dataset):
    def __init__(self):
        self.data = pandas.read_csv('train.csv',header=0)
            
            
        self.ordinals = {}
        for sample in tqdm.tqdm(self.data.text):
            for token in nlp (sample.lower(),disable = ['parser','tagger','ner']):
                if token.text not in self.ordinals:
                    self.ordinals[token.text] = len(self.ordinals)
    def __len__ (self):
        return len(self.data)
    def __getitem__(self,idx):
        if type(idx) is torch.Tensor:
            idx = idx.item()
        sample = self.data.iloc[idx]
        bag_of_words = torch.zeros(len(self.ordinals))
        for token in nlp (sample.text.lower(),disable = ['parser','tagger','ner']):
            bag_of_words[self.ordinals[token.text]] += 1
        return bag_of_words, torch.tensor(sample.target)
traindataset = TrainDataset()
traindataset[0]

100%|██████████| 7613/7613 [00:01<00:00, 4181.32it/s]


(tensor([1., 1., 1.,  ..., 0., 0., 0.]), tensor(1))

In [4]:
class TestDataset(Dataset):
    def __init__(self):
        self.data = pandas.read_csv('test.csv',header=0)
            
            
        self.ordinals = {}
        for sample in tqdm.tqdm(self.data.text):
            for token in nlp (sample.lower(),disable = ['parser','tagger','ner']):
                if token.text not in self.ordinals:
                    self.ordinals[token.text] = len(self.ordinals)
        #print('efsfgsdfsdf',len(self.ordinals))
    def __len__ (self):
        return len(self.data)
    def __getitem__(self,idx):
        if type(idx) is torch.Tensor:
            idx = idx.item()
        sample = self.data.iloc[idx]
        bag_of_words = torch.zeros(22610)
        for token in nlp (sample.text.lower(),disable = ['parser','tagger','ner']):
            bag_of_words[self.ordinals[token.text]] += 1
        return bag_of_words
testdataset = TestDataset()
testdataset[0]

100%|██████████| 3263/3263 [00:00<00:00, 4456.60it/s]


tensor([1., 1., 1.,  ..., 0., 0., 0.])

In [5]:
len(traindataset.ordinals)

22610

In [6]:
len(testdataset.ordinals)

12711

In [8]:
number_for_validating = int(len(traindataset)*0.1)
number_for_training = len(traindataset) - number_for_validating
validation,train = torch.utils.data.random_split(traindataset,[
    number_for_validating,number_for_training])
trainloader = torch.utils.data.DataLoader(train,batch_size =16, shuffle =True)
validationloader = torch.utils.data.DataLoader(validation,batch_size =16, shuffle =True)
print(len(validation),len(train),len(trainloader),len(validationloader))

761 6852 429 48


In [9]:
test_number = int(len(testdataset))
print(test_number)
test_loader = torch.utils.data.DataLoader(testdataset,batch_size =16, shuffle =True)

3263


In [0]:
class Modeltrain (torch.nn.Module):
    def __init__(self, input_dimensions,size=128):
        super().__init__()
        self.layer_one = torch.nn.Linear(input_dimensions, size)
        self.activation_one = torch.nn.ReLU()
        self.layer_two = torch.nn.Linear(size, size)
        self.activation_two = torch.nn.ReLU()
        self.shape_outputs = torch.nn.Linear(size, 2)

    def forward(self, inputs):

        buffer = self.layer_one(inputs)
        buffer = self.activation_one(buffer)
        buffer = self.layer_two(buffer)
        buffer = self.activation_two(buffer)
        buffer = self.shape_outputs(buffer)
        return buffer
modeltrain = Modeltrain(len(traindataset.ordinals))

In [12]:
optimizer = torch.optim.Adam(modeltrain.parameters())
loss_function = torch.nn.CrossEntropyLoss()
modeltrain.train()
for epoch in range(16):
    losses = []
    for inputs, outputs in tqdm.tqdm(trainloader):
        optimizer.zero_grad()
        results = modeltrain(inputs)
        loss = loss_function(results, outputs)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
    print("Loss: {0}".format(torch.tensor(losses).mean()))

  0%|          | 2/429 [00:00<00:25, 16.97it/s]

Loss: 0.5005127787590027


  0%|          | 2/429 [00:00<00:29, 14.51it/s]

Loss: 0.2223334163427353


  0%|          | 2/429 [00:00<00:31, 13.58it/s]

...

Loss: 0.01235273852944374


  0%|          | 2/429 [00:00<00:32, 13.34it/s]

Loss: 0.012400202453136444


100%|██████████| 429/429 [00:34<00:00, 12.37it/s]

Loss: 0.012907399795949459


In [14]:
results_buffer = []
actual_buffer = []
with torch.no_grad():
    modeltrain.eval()

    for inputs, actual in validationloader:
        results = modeltrain(inputs).argmax(dim=1).numpy()
        results_buffer.append(results)
        actual_buffer.append(actual)
print(sklearn.metrics.classification_report(
    np.concatenate(actual_buffer),
    np.concatenate(results_buffer)))

              precision    recall  f1-score   support

           0       0.77      0.81      0.79       444
           1       0.72      0.66      0.69       317

    accuracy                           0.75       761
   macro avg       0.74      0.74      0.74       761
weighted avg       0.75      0.75      0.75       761



In [15]:
results= []
with torch.no_grad():
    modeltrain.eval()

    for inputs in test_loader:
        result = modeltrain(inputs).argmax(dim=1).numpy()
        results.append(result)
results = np.hstack(results)
print(results.shape)

(3263,)


In [0]:
np.savetxt("foo.csv", results.astype(int), fmt='%d')